In [1]:
import pandas as pd
import sqlite3
import os

# Conexão com o banco de dados
db_path = 'walmart_fraudes.db'
if not os.path.exists(db_path):
    db_path = r'C:\Users\louis\datatech\Database\walmart_fraudes.db'

conn = sqlite3.connect(db_path)

# Consultas para o dashboard
queries = {
    # 1. Resumo geral de fraudes por região
    'fraudes_por_regiao': """
    SELECT region, 
           COUNT(*) as total_pedidos,
           SUM(items_missing) as total_itens_faltantes,
           ROUND(AVG(items_missing), 2) as media_itens_faltantes,
           ROUND(SUM(items_missing) * 100.0 / SUM(items_delivered + items_missing), 2) as percentual_fraude
    FROM orders
    GROUP BY region
    ORDER BY percentual_fraude DESC
    """,
    
    # 2. Motoristas com maior índice de fraude
    'motoristas_suspeitos': """
    SELECT d.driver_id, d.driver_name, d.age,
           COUNT(o.order_id) as total_entregas,
           SUM(o.items_missing) as itens_faltantes,
           ROUND(AVG(o.items_missing), 2) as media_itens_faltantes,
           ROUND(SUM(o.items_missing) * 100.0 / SUM(o.items_delivered + o.items_missing), 2) as percentual_fraude
    FROM orders o
    JOIN drivers d ON o.driver_id = d.driver_id
    GROUP BY d.driver_id
    HAVING total_entregas > 5
    ORDER BY percentual_fraude DESC
    LIMIT 50
    """,
    
    # 3. Clientes com maior índice de fraude
    'clientes_suspeitos': """
    SELECT c.customer_id, c.customer_name, c.customer_age,
           COUNT(o.order_id) as total_pedidos,
           SUM(o.items_missing) as itens_faltantes,
           ROUND(AVG(o.items_missing), 2) as media_itens_faltantes,
           ROUND(SUM(o.items_missing) * 100.0 / SUM(o.items_delivered + o.items_missing), 2) as percentual_fraude
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    GROUP BY c.customer_id
    HAVING total_pedidos > 3
    ORDER BY percentual_fraude DESC
    LIMIT 50
    """,
    
    # 4. Análise de horário das fraudes
    'fraudes_por_horario': """
    SELECT 
        delivery_hour_only as hora,
        period_of_day as periodo_dia,
        COUNT(*) as total_pedidos,
        SUM(CASE WHEN items_missing > 0 THEN 1 ELSE 0 END) as pedidos_com_fraude,
        ROUND(SUM(CASE WHEN items_missing > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as percentual_fraude
    FROM orders
    GROUP BY delivery_hour_only
    ORDER BY delivery_hour_only
    """,
    
    # 5. Produtos mais reportados como não entregues
    'produtos_nao_entregues': """
    SELECT 
        p.product_id,
        p.product_name,
        p.category,
        p.price,
        COUNT(DISTINCT CASE WHEN m.product_id_1 = p.product_id THEN m.order_id ELSE NULL END) +
        COUNT(DISTINCT CASE WHEN m.product_id_2 = p.product_id THEN m.order_id ELSE NULL END) +
        COUNT(DISTINCT CASE WHEN m.product_id_3 = p.product_id THEN m.order_id ELSE NULL END) as total_relatos
    FROM products p
    LEFT JOIN missing_items m ON 
        p.product_id = m.product_id_1 OR
        p.product_id = m.product_id_2 OR
        p.product_id = m.product_id_3
    GROUP BY p.product_id
    HAVING total_relatos > 0
    ORDER BY total_relatos DESC
    LIMIT 50
    """,
    
    # 6. Tendência temporal de fraudes
    'tendencia_fraudes': """
    SELECT 
        date,
        COUNT(*) as total_pedidos,
        SUM(items_missing) as itens_faltantes,
        ROUND(SUM(items_missing) * 100.0 / SUM(items_delivered + items_missing), 2) as percentual_fraude
    FROM orders
    GROUP BY date
    ORDER BY date
    """
}

# Criar diretório para exportar os csvs
output_dir = 'dashboard_data'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Executar consultas e exportar para CSV
for name, query in queries.items():
    df = pd.read_sql_query(query, conn)
    csv_path = os.path.join(output_dir, f'{name}.csv')
    df.to_csv(csv_path, index=False)
    print(f"Arquivo '{name}.csv' gerado com sucesso!")

# Consultas adicionais para modelos de detecção de anomalias
anomaly_query = """
SELECT
    d.driver_id,
    d.driver_name,
    d.age,
    d.Trips,
    COUNT(o.order_id) as orders_delivered,
    AVG(o.items_missing) as avg_missing_items,
    SUM(o.items_missing) as total_missing_items,
    SUM(o.items_delivered) as total_delivered_items,
    SUM(o.items_delivered + o.items_missing) as total_items,
    CAST(SUM(o.items_missing) AS FLOAT) / 
        CAST(SUM(o.items_delivered + o.items_missing) AS FLOAT) as missing_ratio,
    AVG(o.order_amount) as avg_order_amount,
    SUM(CASE WHEN o.items_missing > 0 THEN 1 ELSE 0 END) as orders_with_missing,
    CAST(SUM(CASE WHEN o.items_missing > 0 THEN 1 ELSE 0 END) AS FLOAT) / 
        CAST(COUNT(o.order_id) AS FLOAT) as problem_order_ratio
FROM drivers d
JOIN orders o ON d.driver_id = o.driver_id
GROUP BY d.driver_id
"""
driver_anomalies = pd.read_sql_query(anomaly_query, conn)
driver_anomalies.to_csv(os.path.join(output_dir, 'driver_features_model.csv'), index=False)

# Fechar conexão
conn.close()

print("\nTodos os dados para o dashboard foram exportados com sucesso!")

Arquivo 'fraudes_por_regiao.csv' gerado com sucesso!
Arquivo 'motoristas_suspeitos.csv' gerado com sucesso!
Arquivo 'clientes_suspeitos.csv' gerado com sucesso!
Arquivo 'fraudes_por_horario.csv' gerado com sucesso!
Arquivo 'produtos_nao_entregues.csv' gerado com sucesso!
Arquivo 'tendencia_fraudes.csv' gerado com sucesso!

Todos os dados para o dashboard foram exportados com sucesso!
